In [1]:
import csv
import pandas as pd
import holidays
# 定义文件名
filename = "/Users/eshoyuan/Code/Other/CU DS Hackthon/MTA.csv"

# 初始化集合来存储结果
df = pd.read_csv(filename) # 选择你需要的列 selected_columns = ['transit_timestamp', 'station_complex_id', 'routes', 'payment_method', 'ridership', 'transfers', 'weather'] df_selected = df[selected_columns]
selected_columns = ['transit_timestamp', 'station_complex_id', 'routes', 'payment_method', 'ridership', 'transfers', 'weather']
df_selected = df[selected_columns]
df_selected['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

df_selected['weekday'] = df_selected['transit_timestamp'].dt.day_name()
us_holidays = holidays.US(years=[2000, 2023])
df_selected['is_holiday'] = df_selected['transit_timestamp'].dt.date.apply(lambda x: x in us_holidays)
df_selected["routes"] = df_selected["routes"].str.split(",")



/var/folders/q8/bq8zhq5d6gv49nv9s43z71z80000gn/T/ipykernel_27970/2757360441.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])
/var/folders/q8/bq8zhq5d6gv49nv9s43z71z80000gn/T/ipykernel_27970/2757360441.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['weekday'] = df_selected['transit_timestamp'].dt.day_name()
/var/folders/q8/bq8zhq5d6gv49nv9s43z71z80000gn/T/ipykernel_27970/2757360441.py:15: SettingWithCopyWarning: 

In [2]:
df_train = df_selected[df_selected['transit_timestamp'] <= '2023-05-01']
df_test = df_selected[df_selected['transit_timestamp'] > '2023-05-01']

In [3]:
# 对'routes'列进行展开，每行只包含一个路线
df_expanded = df_test.explode('routes')

# 从'transit_timestamp'列中提取小时
df_expanded['hour'] = df_expanded['transit_timestamp'].dt.hour
df_expanded = df_expanded[df_expanded['routes'].notna() & (df_expanded['routes'] != '')]

# 定义一个函数来计算每组的统计数据
def calculate_group(group):
    omny_mask = group['payment_method'] == 'omny'
    metrocard_mask = group['payment_method'] == 'metrocard'
    
    data = {
        'omny_ridership_sum': group[omny_mask]['ridership'].sum(),
        'omny_transfers_sum': group[omny_mask]['transfers'].sum(),
        'omny_count': omny_mask.sum(),
        'mastercard_ridership_sum': group[metrocard_mask]['ridership'].sum(),
        'mastercard_transfers_sum': group[metrocard_mask]['transfers'].sum(),
        'mastercard_count': metrocard_mask.sum(),
    }
    
    return pd.Series(data)

# 使用groupby方法并应用上述函数
grouped = df_expanded.groupby(['routes', 'hour']).apply(calculate_group).reset_index()

# 修改列名称，以便与新数据框保持一致
grouped.rename(columns={'routes': 'route'}, inplace=True)

# 结果应该与你的new_df数据框结构相似
grouped["omny_ridership"] = grouped["omny_ridership_sum"] / grouped["omny_count"]
grouped["omny_transfers"] = grouped["omny_transfers_sum"] / grouped["omny_count"]
grouped["mastercard_ridership"] = grouped["mastercard_ridership_sum"] / grouped["mastercard_count"]
grouped["mastercard_transfers"] = grouped["mastercard_transfers_sum"] / grouped["mastercard_count"]
grouped["route"] = grouped["route"].apply(lambda x: str(x))

In [4]:
grouped

,route,hour,omny_ridership_sum,omny_transfers_sum,omny_count,mastercard_ridership_sum,mastercard_transfers_sum,mastercard_count,omny_ridership,omny_transfers,mastercard_ridership,mastercard_transfers
0,1,0,1434300,42191,5621,1470715,21740,5696,255.168120,7.505960,258.201369,3.816713
1,1,1,632328,17842,5294,580243,6133,5633,119.442388,3.370230,103.007811,1.088763
2,1,2,308434,9235,5034,324909,2939,5535,61.270163,1.834525,58.700813,0.530985
3,1,3,171627,5254,4996,249054,3538,5535,34.352882,1.051641,44.996206,0.639205
4,1,4,192651,6791,5347,425903,8933,5636,36.029736,1.270058,75.568311,1.584989
...,...,...,...,...,...,...,...,...,...,...,...,...
571,Z,19,2347259,44881,2950,2104993,62273,2951,795.681017,15.213898,713.315147,21.102338
572,Z,20,1829292,31951,2945,1623176,46589,2952,621.151783,10.849236,549.856369,15.782182
573,Z,21,1480022,27304,2944,1259807,36505,2953,502.724864,9.274457,426.619370,12.362005
574,Z,22,1228807,18880,2906,999482,25657,2945,422.851686,6.496903,339.382683,8.712054


In [5]:
# 对'routes'列进行展开，每行只包含一个路线
df_expanded = df_test.explode('routes')

# 从'transit_timestamp'列中提取小时
df_expanded['hour'] = df_expanded['transit_timestamp'].dt.hour
df_expanded = df_expanded[df_expanded['routes'].notna() & (df_expanded['routes'] != '')]

# 定义一个函数来计算每组的统计数据
def calculate_group(group):
    
    
    data = {
        'total_ride': group['ridership'].sum() + group['transfers'].sum(),
        'weather': group['weather'].iloc[0],  # 假设每组内的weather, weekday和is_holiday是一致的
        'weekday': group['weekday'].iloc[0],
        'is_holiday': group['is_holiday'].iloc[0],
        "hour": group["hour"].iloc[0]
    }
    
    return pd.Series(data)

# 使用groupby方法并应用上述函数
grouped2 = df_expanded.groupby(['routes', 'transit_timestamp']).apply(calculate_group).reset_index()

# 修改列名称，以便与新数据框保持一致
grouped2.rename(columns={'routes': 'route'}, inplace=True)
grouped2["route"] = grouped2["route"].apply(lambda x: str(x))

grouped2

,route,transit_timestamp,total_ride,weather,weekday,is_holiday,hour
0,1,2023-05-01 01:00:00,1363,Mist,Monday,False,1
1,1,2023-05-01 02:00:00,714,Rain,Monday,False,2
2,1,2023-05-01 03:00:00,681,Rain,Monday,False,3
3,1,2023-05-01 04:00:00,1466,Clouds,Monday,False,4
4,1,2023-05-01 05:00:00,7291,Rain,Monday,False,5
...,...,...,...,...,...,...,...
87570,Z,2023-09-29 20:00:00,15354,Rain,Friday,False,20
87571,Z,2023-09-29 21:00:00,11430,Rain,Friday,False,21
87572,Z,2023-09-29 22:00:00,10716,Rain,Friday,False,22
87573,Z,2023-09-29 23:00:00,8211,Rain,Friday,False,23


In [6]:
df_test = pd.merge(grouped2, grouped, on=['route', 'hour'], how='left')
df_test

,route,transit_timestamp,total_ride,weather,weekday,is_holiday,hour,omny_ridership_sum,omny_transfers_sum,omny_count,mastercard_ridership_sum,mastercard_transfers_sum,mastercard_count,omny_ridership,omny_transfers,mastercard_ridership,mastercard_transfers
0,1,2023-05-01 01:00:00,1363,Mist,Monday,False,1,632328,17842,5294,580243,6133,5633,119.442388,3.370230,103.007811,1.088763
1,1,2023-05-01 02:00:00,714,Rain,Monday,False,2,308434,9235,5034,324909,2939,5535,61.270163,1.834525,58.700813,0.530985
2,1,2023-05-01 03:00:00,681,Rain,Monday,False,3,171627,5254,4996,249054,3538,5535,34.352882,1.051641,44.996206,0.639205
3,1,2023-05-01 04:00:00,1466,Clouds,Monday,False,4,192651,6791,5347,425903,8933,5636,36.029736,1.270058,75.568311,1.584989
4,1,2023-05-01 05:00:00,7291,Rain,Monday,False,5,756058,44484,5630,1532527,72300,5672,134.290941,7.901243,270.191643,12.746827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87570,Z,2023-09-29 20:00:00,15354,Rain,Friday,False,20,1829292,31951,2945,1623176,46589,2952,621.151783,10.849236,549.856369,15.782182
87571,Z,2023-09-29 21:00:00,11430,Rain,Friday,False,21,1480022,27304,2944,1259807,36505,2953,502.724864,9.274457,426.619370,12.362005
87572,Z,2023-09-29 22:00:00,10716,Rain,Friday,False,22,1228807,18880,2906,999482,25657,2945,422.851686,6.496903,339.382683,8.712054
87573,Z,2023-09-29 23:00:00,8211,Rain,Friday,False,23,780785,10223,2831,664789,13809,2930,275.798304,3.611091,226.890444,4.712969


In [7]:
selected_columns = ["route", "hour", "total_ride", "weather", "weekday", "is_holiday", "omny_ridership", "omny_transfers", "mastercard_ridership", "mastercard_transfers"]
df_test = df_test[selected_columns]
df_test

,route,hour,total_ride,weather,weekday,is_holiday,omny_ridership,omny_transfers,mastercard_ridership,mastercard_transfers
0,1,1,1363,Mist,Monday,False,119.442388,3.370230,103.007811,1.088763
1,1,2,714,Rain,Monday,False,61.270163,1.834525,58.700813,0.530985
2,1,3,681,Rain,Monday,False,34.352882,1.051641,44.996206,0.639205
3,1,4,1466,Clouds,Monday,False,36.029736,1.270058,75.568311,1.584989
4,1,5,7291,Rain,Monday,False,134.290941,7.901243,270.191643,12.746827
...,...,...,...,...,...,...,...,...,...,...
87570,Z,20,15354,Rain,Friday,False,621.151783,10.849236,549.856369,15.782182
87571,Z,21,11430,Rain,Friday,False,502.724864,9.274457,426.619370,12.362005
87572,Z,22,10716,Rain,Friday,False,422.851686,6.496903,339.382683,8.712054
87573,Z,23,8211,Rain,Friday,False,275.798304,3.611091,226.890444,4.712969


In [8]:
df_test.to_csv('test.csv', index=False)
